# NER 序列标注

## baseline：bert + softmax

In [9]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from transformers import BertForTokenClassification
from transformers import BertTokenizerFast

### 加载数据

In [10]:
def load_data():
    file_train = '../dataset/msra_ner/train/part.txt'
    df = pd.read_csv(file_train, sep='\t', nrows=1000, header=None)
    df.columns = ['text', 'labels']
    df['text'] = df['text'].apply(lambda x: x.replace('', ' '))
    df['labels'] = df['labels'].apply(lambda x: x.replace('', ' '))

    return df

df = load_data()
df.head(10)

,text,labels
0,海 钓 比 赛 地 点 在 厦 门 与 金 门 之 间 的 海 域 。,O O O O O O O B-LOC I-LOC O B-LOC I-LOC O O O ...
1,这 座 依 山 傍 水 的 博 物 馆 由 国 内 一 流 的 设 计 师 主 持 设 计 ...,O O O O O O O O O O O O O O O O O O O O O O O ...
2,但 作 为 一 个 共 产 党 员 、 人 民 公 仆 ， 应 当 胸 怀 宽 阔 ， 真 ...,O O O O O O O O O O O O O O O O O O O O O O O ...
3,在 发 达 国 家 ， 急 救 保 险 十 分 普 及 ， 已 成 为 社 会 保 障 体 ...,O O O O O O O O O O O O O O O O O O O O O O O ...
4,日 俄 两 国 国 内 政 局 都 充 满 变 数 ， 尽 管 日 俄 关 系 目 前 是 ...,B-LOC B-LOC O O O O O O O O O O O O O O B-LOC ...
5,克 马 尔 的 女 儿 让 娜 今 年 读 五 年 级 ， 她 所 在 的 班 上 有 3 ...,B-PER I-PER I-PER O O O B-PER I-PER O O O O O ...
6,参 加 步 行 的 有 男 有 女 ， 有 年 轻 人 ， 也 有 中 年 人 。,O O O O O O O O O O O O O O O O O O O O O
7,沙 特 队 教 练 佩 雷 拉 ： 两 支 队 都 想 胜 ， 因 此 都 作 出 了 最 ...,B-ORG I-ORG I-ORG O O B-PER I-PER I-PER O O O ...
8,这 种 混 乱 局 面 导 致 有 些 海 域 使 用 者 的 合 法 权 益 难 以 得 ...,O O O O O O O O O O O O O O O O O O O O O O O ...
9,鲁 宾 明 确 指 出 ， 对 政 府 的 这 种 指 控 完 全 没 有 事 实 根 据 ...,B-PER I-PER O O O O O O O O O O O O O O O O O ...


### label2id, id2label

In [11]:
def make_label_id_dict(df):
    labels = [x.split() for x in df.labels.values.tolist()]
    # 标签去重
    unique_labels = set()
    for lb in labels:
        [unique_labels.add(i) for i in lb]
        
    print(unique_labels)
        
    label2id = {v: k for k, v in enumerate(unique_labels)}
    id2label = {k: v for k, v in enumerate(unique_labels)}
    
    return label2id, id2label, unique_labels, labels

label2id, id2label, unique_labels, labels = make_label_id_dict(df)

print(label2id)
print('-' * 20)
print(id2label)

{'I-LOC', 'I-PER', 'I-ORG', 'B-PER', 'O', 'B-ORG', 'B-LOC'}
{'I-LOC': 0, 'I-PER': 1, 'I-ORG': 2, 'B-PER': 3, 'O': 4, 'B-ORG': 5, 'B-LOC': 6}
--------------------
{0: 'I-LOC', 1: 'I-PER', 2: 'I-ORG', 3: 'B-PER', 4: 'O', 5: 'B-ORG', 6: 'B-LOC'}


### tokenizer化

In [12]:
model_path = "../../../models/bert-base-chinese/"

In [13]:
tokenizer = BertTokenizerFast.from_pretrained(model_path)

In [14]:
# 测试
text = df['text'].values.tolist()
example = text[0]
text_tokenized = tokenizer(example, 
                           padding='max_length', 
                           max_length=50, 
                           truncation=True,
                           return_tensors="pt")
print(example)
print('-' * 40)
print(text_tokenized)
print('-' * 40)
# 解码为原始text
print(tokenizer.decode(text_tokenized.input_ids[0]))

print('-' * 40)
# NOTE:tokenizer后可能有以下问题：
    # BERT 分词器在底层使用了所谓的 word-piece tokenizer，它是一个子词分词器。这意味着 BERT tokenizer 可能会将一个词拆分为一个或多个有意义的子词
    # 导致tokenizer后的序列长度和原始label长度不一致
    # 需要调整标签，以达到一一对应的结果。使其与标记化后的序列具有相同的长度

print(tokenizer.convert_ids_to_tokens(text_tokenized["input_ids"][0]))
print('-' * 40)

# 使用word_ids来调整
# 每个拆分的 token 共享相同的 word_ids，其中来自 BERT 的特殊 token，例如 [CLS]、[SEP] 和 [PAD] 都没有特定word_ids的，结果是None
word_ids = text_tokenized.word_ids()
print(word_ids)

海 钓 比 赛 地 点 在 厦 门 与 金 门 之 间 的 海 域 。
----------------------------------------
{'input_ids': tensor([[ 101, 3862, 7157, 3683, 6612, 1765, 4157, 1762, 1336, 7305,  680, 7032,
         7305,  722, 7313, 4638, 3862, 1818,  511,  102,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]])}
----------------------------------------
[CLS] 海 钓 比 赛 地 点 在 厦 门 与 金 门 之 间 的 海 域 。 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

In [15]:

# 方法1：只为每个拆分token的第一个子词提供一个标签。子词的延续将简单地用"-100"作为标签。所有没有word_ids 的token也将标为 "-100"。
# 方法2：在属于同一 token 的所有子词中提供相同的标签。所有没有word_ids的token都将标为 "-100"。
# """
def align_label_example(tokenized_input, labels, label_all_tokens, labels_to_ids):
    word_ids = tokenized_input.word_ids()
    pre_word_idx = None
    label_ids = []
    for word_idx in word_ids:
        # 把特殊token的label置为-100
        if word_idx is None:
            label_ids.append(-100)
        # 当前id是新的词或者单词的第一个子词
        elif word_idx != pre_word_idx:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]])
            except:
                label_ids.append(-100)
        else:
            # 当前id是上一个词的子词
            label_ids.append(labels_to_ids[labels[word_idx]] if label_all_tokens else -100)
        pre_word_idx = word_idx
    return label_ids

label = labels[0]
print(label)
print('-' * 40)
new_label = align_label_example(text_tokenized, label, True, label2id)
print(new_label)
print('-' * 40)
print(tokenizer.convert_ids_to_tokens(text_tokenized["input_ids"][0]))

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O']
----------------------------------------
[-100, 4, 4, 4, 4, 4, 4, 4, 6, 0, 4, 6, 0, 4, 4, 4, 4, 4, 4, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]
----------------------------------------
['[CLS]', '海', '钓', '比', '赛', '地', '点', '在', '厦', '门', '与', '金', '门', '之', '间', '的', '海', '域', '。', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


### dataset构建

In [16]:
label_all_tokens = True

# 创建一个Dataset类来批量生成和获取数据


def align_label(texts, labels):
    # 首先tokenizer输入文本
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)
    # 获取word_ids
    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []
    # 采用上述的第2种方法来调整标签，使得标签与输入数据对其。
    for word_idx in word_ids:
        # 如果token不在word_ids内，则用 “-100” 填充
        if word_idx is None:
            label_ids.append(-100)
        # 如果token在word_ids内，且word_idx不为None，则从labels_to_ids获取label id
        elif word_idx != previous_word_idx:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]])
            except:
                label_ids.append(-100)
        # 和上一个属于同一个词，则则从labels_to_ids获取label id
        else:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]] if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids


# 构建自己的数据集类
class DataSequence(torch.utils.data.Dataset):
    def __init__(self, df):
        lb = [i.split() for i in df['labels'].values.tolist()]
        # tokenizer 向量化文本
        txt = df['text'].values.tolist()
        self.texts = [tokenizer(str(i), 
                                padding='max_length', 
                                max_length = 512, 
                                truncation=True, 
                                return_tensors="pt") for i in txt]
        # 对齐标签
        self.labels = [align_label(i,j) for i,j in zip(txt, lb)]

    def __len__(self):
        return len(self.labels)

    def get_batch_data(self, idx):
        return self.texts[idx]

    def get_batch_labels(self, idx):
        return torch.LongTensor(self.labels[idx])

    def __getitem__(self, idx):
        label = torch.tensor(self.labels[idx])
        text = self.texts[idx]
        
        return text, label


In [17]:
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                            [int(.8 * len(df)), int(.9 * len(df))])

print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(800, 2)
(100, 2)
(100, 2)


### 构建模型

In [18]:
class BertNer(torch.nn.Module):
    def __init__(self):
        super(BertNer, self).__init__()
        
        # BertForTokenClassification模型是针对序列标注任务进行优化，默认损失函数是交叉熵损失函数
        # num_labels：分类任务的类别数量
        self.bert = BertForTokenClassification.from_pretrained(model_path,
                                                               num_labels=len(unique_labels))

    def forward(self, input_id, mask, label):
        output = self.bert(input_ids=input_id, 
                           attention_mask=mask,
                           labels=label, 
                           return_dict=False)
        return output

### 模型训练

In [19]:
LEARNING_RATE = 1e-2
EPOCHS = 1
model = BertNer()

from transformers import AdamW, get_linear_schedule_with_warmup

#!pip install tdqm


Some weights of BertForTokenClassification were not initialized from the model checkpoint at ../../../models/bert-base-chinese/ and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
train_dataset = DataSequence(df_train)
val_dataset = DataSequence(df_val)

print(len(train_dataset))
print(train_dataset[0][0].keys())
print('-' * 40)
print(train_dataset[0][1].shape)

800
dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
----------------------------------------
torch.Size([512])


In [27]:
# 定义训练和验证集数据
train_dataset = DataSequence(df_train)
val_dataset = DataSequence(df_val)
# 批量获取训练和验证集数据
train_dataloader = DataLoader(train_dataset, num_workers=1, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, num_workers=1, batch_size=8)
# 判断是否使用GPU，如果有，尽量使用，可以加快训练速度
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
# 定义优化器
# optimizer = SGD(model.parameters(), lr=LEARNING_RATE)
total_steps = len(train_dataloader) * EPOCHS
optim = AdamW(model.parameters(), lr=2e-5)
scheduler = get_linear_schedule_with_warmup(optim,
                                       num_warmup_steps=0,
                                       num_training_steps=total_steps)

D:\Python\envs\py310\lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
print(len(train_dataloader))
for x, y in train_dataloader:
    train_label = train_label[0]
    mask = train_data['attention_mask'][0]
    input_id = train_data['input_ids'][0]

    print(train_label.shape)
    print(mask.shape)
    print(input_id.shape)

    a=asas

100


In [14]:
def train_loop(model, df_train, df_val):
    if use_cuda:
        model = model.cuda()
    # 开始训练循环
    best_acc = 0
    best_loss = 1000
    for epoch_num in range(EPOCHS):
        total_acc_train = 0
        total_loss_train = 0
        # 训练模型
        model.train()
        # 按批量循环训练模型
        for train_data, train_label in train_dataloader:
              # 从train_data中获取mask和input_id
            train_label = train_label[0].to(device)
            mask = train_data['attention_mask'][0].to(device)
            input_id = train_data['input_ids'][0].to(device)
            
            # 梯度清零！！
            optim.zero_grad()
            # 输入模型训练结果：损失及分类概率
            loss, logits = model(input_id, mask, train_label)
            # 过滤掉特殊token及padding的token
            logits_clean = logits[0][train_label != -100]
            label_clean = train_label[train_label != -100]
            # 获取最大概率值
            predictions = logits_clean.argmax(dim=1)
      # 计算准确率
            acc = (predictions == label_clean).float().mean()
            total_acc_train += acc
            total_loss_train += loss.item()
      # 反向传递
            loss.backward()
            # 参数更新
            optim.step()
            scheduler.step()
        # 模型评估
        model.eval()

        total_acc_val = 0
        total_loss_val = 0
        for val_data, val_label in val_dataloader:
      # 批量获取验证数据
            val_label = val_label[0].to(device)
            mask = val_data['attention_mask'][0].to(device)
            input_id = val_data['input_ids'][0].to(device)
      # 输出模型预测结果
            loss, logits = model(input_id, mask, val_label)
      # 清楚无效token对应的结果
            logits_clean = logits[0][val_label != -100]
            label_clean = val_label[val_label != -100]
            # 获取概率值最大的预测
            predictions = logits_clean.argmax(dim=1)          
            # 计算精度
            acc = (predictions == label_clean).float().mean()
            total_acc_val += acc
            total_loss_val += loss.item()

        val_accuracy = total_acc_val / len(df_val)
        val_loss = total_loss_val / len(df_val)

        print(
            f'''Epochs: {epoch_num + 1} | 
                Loss: {total_loss_train / len(df_train): .3f} | 
                Accuracy: {total_acc_train / len(df_train): .3f} |
                Val_Loss: {total_loss_val / len(df_val): .3f} | 
                Accuracy: {total_acc_val / len(df_val): .3f}''')

In [ ]:
train_loop(model, df_train, df_val)

> 注意，有一个重要的步骤！在训练循环的每个 epoch 中，在模型预测之后，需要忽略所有以 "-100" 作为标签的token

# 评估模型

In [ ]:
def evaluate(model, df_test):
    # 定义测试数据
    test_dataset = DataSequence(df_test)
    # 批量获取测试数据
    test_dataloader = DataLoader(test_dataset, num_workers=4, batch_size=1)
   # 使用GPU
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()
    total_acc_test = 0.0
    for test_data, test_label in test_dataloader:
        test_label = test_label[0].to(device)
        mask = test_data['attention_mask'][0].to(device)
        input_id = test_data['input_ids'][0].to(device)
          
        loss, logits = model(input_id, mask, test_label.long())
        logits_clean = logits[0][test_label != -100]
        label_clean = test_label[test_label != -100]
        predictions = logits_clean.argmax(dim=1)             
        acc = (predictions == label_clean).float().mean()
        total_acc_test += acc
    val_accuracy = total_acc_test / len(df_test)
    print(f'Test Accuracy: {total_acc_test / len(df_test): .3f}')

evaluate(model, df_test)

# 使用经过训练的模型来预测文本或句子中每个单词的实体

In [ ]:
def align_word_ids(texts): 
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)
    word_ids = tokenized_inputs.word_ids()
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                # NOTE：这里的1并不是真是label，只要不是-100即可
                label_ids.append(1)
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(1 if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx
    return label_ids

def evaluate_one_text(model, sentence):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    text = tokenizer(sentence, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")
    mask = text['attention_mask'][0].unsqueeze(0).to(device)
    input_id = text['input_ids'][0].unsqueeze(0).to(device)
    label_ids = torch.Tensor(align_word_ids(sentence)).unsqueeze(0).to(device)
    
    logits = model(input_id, mask, None)
    print(logits)
    # NOTE: 预测时没有label，所以模型只会返回logits，没有loss
    # 去除-100标记
    logits_clean = logits[0][label_ids != -100]
    
    predictions = logits_clean.argmax(dim=1).tolist()
    prediction_label = [ids_to_labels[i] for i in predictions]
    print(sentence)
    print(prediction_label)
            
evaluate_one_text(model, '侯海伦在石家庄的御芝林公司担任算法工程师')